In [ ]:
idp = ['lh_G&S_frontomargin_thickness']
df_te  = pd.DataFrame(columns = ['age','sex', 'site', 'sitenum'], index = ['C1', 'C2', 'C3'])
y_te = np.array([0.065884, 1.897501,1.897501])

In [ ]:
df_te['age'].iloc[0] = 39
df_te['age'].iloc[1] = 30
df_te['age'].iloc[2] = 39

df_te['sex'].iloc[0] = 1
df_te['sex'].iloc[1] = 1
df_te['sex'].iloc[2] = 1

df_te['sitenum'].iloc[0] = 1000
df_te['sitenum'].iloc[1] = 1000
df_te['sitenum'].iloc[2] = 1000

df_te['site'].iloc[0] = 'NUDZ'
df_te['site'].iloc[1] = 'NUDZ'
df_te['site'].iloc[2] = 'NUDZ'

In [ ]:
models_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/models/pokus')
os.chdir(models_dir)

In [ ]:
# which data columns do we wish to use as covariates? 
cols_cov = ['age','sex']

# limits for cubic B-spline basis 
xmin = -5 
xmax = 110

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 7

# extract and save the response variables for the test set
#y_te = df_te[idp].to_numpy()

# save the variables
resp_file_te = os.path.join(models_dir, 'resp_te.txt') 
np.savetxt(resp_file_te, y_te)
    
# configure and save the design matrix
cov_file_te = os.path.join(models_dir, 'cov_bspline_te.txt')


In [ ]:
X_te = create_design_matrix(df_te[cols_cov], 
                            site_ids = df_te['site'],
                            all_sites = site_ids_tr,
                            basis = 'bspline', 
                            xmin = xmin, 
                            xmax = xmax)
np.savetxt(cov_file_te, X_te)



In [ ]:
df_ad['sitenum'] = 1000

In [ ]:
# save the covariates for the adaptation data
X_ad = create_design_matrix(df_ad[cols_cov], 
                            site_ids = df_ad['site'],
                            all_sites = site_ids_tr,
                            basis = 'bspline', 
                            xmin = xmin, 
                            xmax = xmax)
cov_file_ad = os.path.join(models_dir, 'cov_bspline_ad.txt')          
np.savetxt(cov_file_ad, X_ad)



In [ ]:
# save the responses for the adaptation data
resp_file_ad = os.path.join(models_dir, 'resp_ad.txt') 
y_ad = df_ad[idp].to_numpy()
np.savetxt(resp_file_ad, y_ad)

# save the site ids for the adaptation data
sitenum_file_ad = os.path.join(models_dir, 'sitenum_ad.txt') 
site_num_ad = df_ad['sitenum'].to_numpy(dtype=int)
np.savetxt(sitenum_file_ad, site_num_ad)

# save the site ids for the test data 
sitenum_file_te = os.path.join(models_dir, 'sitenum_te.txt')
site_num_te = df_te['sitenum'].to_numpy(dtype=int)
np.savetxt(sitenum_file_te, site_num_te)

# adaptation files are among inputs to adjust the offset 
yhat_te, s2_te, Z = predict(cov_file_te, 
                            alg = 'blr', 
                            respfile = resp_file_te, 
                            model_path = os.path.join(idp_dir,'Models'),
                            adaptrespfile = resp_file_ad,
                            adaptcovfile = cov_file_ad,
                            adaptvargroupfile = sitenum_file_ad,
                            testvargroupfile = sitenum_file_te)

In [ ]:
idp_dir = os.path.join('/home/barbora/Documents/Projects/Normative_Models/ESO/braincharts/models/lifespan_57K_82sites',idp[0])

In [ ]:
###
# Create dummy data
###

# which sex do we want to plot? 
sex = 1 # 1 = male 0 = female
if sex == 1: 
    clr = 'blue'
else:
    clr = 'red'

# limits for cubic B-spline basis 
xmin = -5 
xmax = 110

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
X0_dummy = np.zeros((len(xx), 2))
X0_dummy[:,0] = xx # intercept
X0_dummy[:,1] = sex # sex covariate

# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None, all_sites=site_ids_tr)

# save the dummy covariates
cov_file_dummy = os.path.join(models_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

In [ ]:
###
# Plotting normative models
###
idp = ['lh_G&S_frontomargin_thickness']
idp = idp[0]

sns.set(style='whitegrid')

print('Running IDP',  idp, ':')
idp_dir = os.path.join(pretrained_dir,'models','lifespan_57K_82sites', idp)
idp_cv_dir = os.path.join(models_dir,idp)


# load the true data points V1
v1_yhat_te = yhat_te
v1_s2_te = s2_te
v1_y_te = y
v1_bspline = pd.DataFrame(load_2d('cov_bspline_te.txt'))

#########################################################################
# set up the covariates for the dummy data
print('Making predictions with dummy covariates (for visualisation)')
yhat, s2 = predict(cov_file_dummy, 
                    alg = 'blr', 
                    respfile = None, 
                    model_path = os.path.join(idp_dir,'Models'), 
                    outputsuffix = '_dummy')

# load the normative model
with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
    nm = pickle.load(handle) 

# get the warp and warp parameters
W = nm.blr.warp
warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1] 

# warp dummy predictions to create the plots
med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)

# extract the different variance components to visualise
beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
s2n = 1/beta # variation (aleatoric uncertainty)
s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)
#########################################################################


#  warp predictions for the true data and compute evaluation metrics
v1_med_te = W.warp_predictions(np.squeeze(v1_yhat_te), np.squeeze(v1_s2_te), warp_param)[0]
v1_med_te = v1_med_te[:, np.newaxis]

#print('metrics:', evaluate(y_te, med_te))


###
# Adjusting         
# adjust the data based on the adaptation dataset 
###
#  
# load the adaptation data
y_ad = load_2d(os.path.join(models_dir, 'resp_ad.txt'))
X_ad = load_2d(os.path.join(models_dir, 'cov_bspline_ad.txt'))

# adjust and rescale the data
y_te_rescaled, s2_rescaled = nm.blr.predict_and_adjust(nm.blr.hyp, 
                                                        X_ad,  
                                                        np.squeeze(y_ad), 
                                                        Xs=None, 
                                                        ys=y_te)



In [ ]:

# save transformed predictions so that we don't have to sompute this ever again
v1_y_te_rescaled_f = os.path.join(models_dir,'v1_y_rescaled_'+str(sex)+'.txt')
np.savetxt(v1_y_te_rescaled_f,y_te_rescaled)


# V1 plot the (adjusted) data points
v1_plot = df_te['age'].to_numpy()

plt.figure(figsize=(8,6))

plt.scatter(v1_plot, y_te_rescaled, color='darkgreen', label='V1', alpha = 0.9)

# plot the median of the dummy data
plt.plot(xx, med, clr)

# fill the gaps in between the centiles
junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)
        
# make the width of each centile proportional to the epistemic uncertainty
junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])    
plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

# plot actual centile lines
plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)

plt.xlabel('Age')
plt.ylabel(idp) 
plt.title(idp)
plt.xlim((10,60))
plt.legend()
#plt.savefig(os.path.join('centiles_' + str(sex)),  bbox_inches='tight')
plt.show()
#plt.savefig(os.path.join(images_dir,idp+'_'+str(sex)+'.png'))
#plt.close()



